<a href="https://colab.research.google.com/github/YanzhaoZ/PreBit/blob/main/Language_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob
import re
import nltk
nltk.download('words')

!pip install yfinance
import yfinance as yf
import numpy as np

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
     |████████████████████████████████| 5.5MB 12.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=05517c76565814a57abda609273d62d01ca2c424b564c7b60715ed291508e30d
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


# Loading in data 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#now that I have the whole string for the day, I need to do processing first, before spliting them into chunks. 
words = set(nltk.corpus.words.words())
#I need to add in some crypto currency specific words. like Etherum.
#bascailly all the symbols of crypto currencies, I should add a set here. 
cryptowords=(['crypto','cryptocurrency','bitcoin','btc','etherum','eos','eth','xrp','ltc'])
words.update(cryptowords)

exclusionli=['up','so','if','go'] #keep in view


def processTweet(tweetFeed):
    tweetFeed = tweetFeed.lower()
    #Convert www.* or https?://* pic.twitter.com* to space
    tweetFeed = re.sub('(https:\/\/www\.[\s][\a-zA-Z]*)|(http:\/\/www\.[\s][\a-zA-Z]*)|(https:\/\/[\s][\a-zA-Z]*)|(http:\/\/[\s][\a-zA-Z]*)|(www\.[\s][\a-zA-Z]*)|(https:\/\/[\a-zA-Z]*)|(http:\/\/[\a-zA-Z]*)',' ',tweetFeed)
    tweetFeed = re.sub('pic\.[^\s]*',' ',tweetFeed)
    #removw @sth and #sth
    tweetFeed = re.sub('(@\ [^\s]*)|(#\ [^\s]*)', ' ', tweetFeed)
    #remove rt
    tweetFeed = re.sub('rt[\s]','',tweetFeed)

    tweetFeed = " ".join(w for w in nltk.wordpunct_tokenize(tweetFeed) if w.lower() in words)# or not w.isalpha())#this part takes in number as True.
    #I'm not so sure about this language filter, but let's just roll with it first. 
    tweetFeed = re.sub("[^A-Za-z']+", ' ', tweetFeed)

    #filter out all the token left with len of 1. possibaly 2, defind a exclusion list: up, if, so, 
    tweetFeed = " ".join(w for w in nltk.wordpunct_tokenize(tweetFeed) if len(w) >1 )
    return tweetFeed


In [ ]:
#now let's try to split the text into smaller chunks, which is just taking the tutorial's code
def get_split(text1,length=200, overlap=50):
  l_total = []
  l_parcial = []
  if len(text1.split())//(length-overlap) >0:
    n = len(text1.split())//(length-overlap)
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:length]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*(length-overlap):w*(length-overlap) + length]
      l_total.append(" ".join(l_parcial))
  return l_total

In [ ]:
#next, tie them up, read in csv, combine into chunks, take the dates as the price? 
path = '/content/gdrive/My Drive/Crypto/tweets2019'
all_files = glob.glob(path+'/*.csv')

#let me try to use list1 to store string and list 2 to store dates
li1=[]
li2=[]
for file in all_files:
  df=pd.read_csv(file,sep=',',index_col=0)
  df=df['text'].to_frame()
  rawtxt=[]
  for row in df.text:
    rawtxt.append(row)
  string = " ".join(rawtxt)

  date= file[-14:-4]
  
  li1.append(string)
  li2.append(date)

data=pd.DataFrame({'date':li2, 'text':li1})



In [ ]:
#now, let's process the text with the pre-defined functin. and perhaps add in the label by looking at the price. 
data['clean_txt']=data['text'].apply(processTweet)

#A note for the price, file date is the day after all the tweets inside. So the label should be created, as such that it reflex:
#file date over previous day change. 
#What index should I use/should I use my own file (maybe not, too big/detailed for daily ticker). Let's go with Yahoofinance.
#now, let's create the price change label, return diff, log return diff, and binary label 
start_date ='2018-12-28'
end_date = '2019-04-02'
price = yf.download("BTC-USD", start=start_date, end=end_date)
price_2= price.Close.to_frame()
price['change']=price_2.apply(lambda x: x/x.shift(1)-1)
price['log_change']=price_2.apply(lambda x: np.log(x)-np.log(x.shift(1)))
price['change_label']=price['change'].apply (lambda x: x>0)
price=price.reset_index()
price['date']=price.Date.apply (lambda x : str(x)[:10])

[*********************100%***********************]  1 of 1 completed


In [ ]:
#price.head()

In [ ]:
#now that I have everything, I need to join the price set with my text set, by the date column. 
#I have a problem with datetime type on price frame vs string in text date, which converstion should I use, convert to string?  Let
# us convert datetime to string first. if need to, we can do the otherway in the future. 
# import datetime
# t = datetime.datetime(2012, 2, 23, 0, 0)
# t.strftime('%Y-%m-%d')

frame_x = data.merge(price, how='outer', on=['date'])
#and drop any non-overlapping cells by dropna.
frame_x = frame_x.dropna()


In [ ]:
#frame_x.head()

In [ ]:
#now that I have the frame prepared, I think I should do the text splitting part? 
#I need to decide on the length to use, and embedding to use. 
#let's roll with the 200 length for now. 

frame_x['text_split']=frame_x.clean_txt.apply(get_split)


In [ ]:
frame_x.head()

,date,text,clean_txt,Date,Open,High,Low,Close,Adj Close,Volume,change,log_change,change_label,text_split
0,2019-01-22,"仮想通貨の時価総額 $119,391,276,842 BTC 価格:$3577.41 ドミナ...",btc eth btc bitcoin current price more on btc ...,2019-01-22,3575.081299,3620.746582,3539.721436,3604.577148,3604.577148,5313623556,0.007982,0.007951,True,[btc eth btc bitcoin current price more on btc...
1,2019-03-26,@ WhatsApp to incorporate a # Bitcoin wallet ?...,to incorporate wallet another step towards ado...,2019-03-26,3969.228760,3985.080811,3944.753174,3985.080811,3985.080811,10707678814,0.005554,0.005538,True,[to incorporate wallet another step towards ad...
2,2019-03-27,I was having a chat the other day during a Bit...,was chat the other day during bitcoin beyond l...,2019-03-27,3984.244873,4087.066162,3977.810547,4087.066162,4087.066162,10897131934,0.025592,0.025270,True,[was chat the other day during bitcoin beyond ...
3,2019-01-23,"$2,000,000 worth of # Bitcoin sold at $3,576.4...",worth of sold at wait for the bounce then corn...,2019-01-23,3605.557129,3623.067871,3565.313965,3585.123047,3585.123047,5433755648,-0.005397,-0.005412,False,[worth of sold at wait for the bounce then cor...
4,2019-01-09,$ BTC | # BTC - bitcoin's Current Price: ▼ $40...,btc bitcoin current price more on stick to bit...,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,4035.296387,5115905224,0.001104,0.001103,True,[btc bitcoin current price more on stick to bi...


In [ ]:
frame_x.shape

(90, 14)

In [ ]:
#ok, i'm gonna rank the frame here by the time entry. 
frame_x= frame_x.sort_values('date',ascending=True)

In [ ]:
#try to save the processed frame to load directly next time
df_save=frame_x[['date','text_split','change_label']]

In [ ]:
df_save.head()

,date,text_split,change_label
71,2019-01-01,[bitcoin could revolutionize governance accord...,True
62,2019-01-02,[btc over now bitcoin cash pay the up threaten...,True
56,2019-01-03,[btc bitcoin price in on resistance at will th...,False
81,2019-01-04,[na data de era do da btc san bitcoin how bitc...,True
73,2019-01-05,[tell my year old self to act less edgy and to...,False


In [ ]:
#ok, now, let's save another version, with every chunk as an entry. 
#this part, omits the dates signal for now. btw 
train_l = []
label_l = []
index_l =[]
for idx,row in df_save.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['change_label'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(23170, 23170, 23170)

In [ ]:
#converting the label from True/False to 1/0. might be redundant, see how. 
label_l_num= [1 if row == True else 0 for row in label_l]

train_df = pd.DataFrame({'text_split':train_l, 'change_label':label_l_num})

#ok, I'm going to split my dataframe into train/validation by a percentage
frame_len= train_df.shape[0]
split_ratio= 0.8
num = round(frame_len * split_ratio)

train_df[:num].to_csv('/content/gdrive/My Drive/Crypto/train_save.csv',index=False)
train_df[num:].to_csv('/content/gdrive/My Drive/Crypto/val_save.csv',index=False)


In [ ]:
train_df[:num].head()

,text_split,change_label
0,bitcoin could revolutionize governance accordi...,1
1,be sparkling clean with all the wash trading g...,1
2,the in the last bitcoin price did not behave a...,1
3,above it for bitcoin on track to replace gold ...,1
4,la de ser en global network in helping those w...,1


In [ ]:
#train_df[num:].change_label.value_counts()
train_df[num:].change_label.sum()/len(train_df[num:])

0.5837289598618903

In [ ]:
!ls '/content/gdrive/My Drive/Crypto'

df_save.csv	      GoogleNews-vectors-negative300.bin.gz
final_price_2019.csv  tweets2019


# Get word embeddings

In [ ]:
#ok text and labels are somewhat prepared, time to bring out the big guns,get the embedding. 
#USE PRETRAINED ONES, what embeddings to use. doesn't have to be BERT. 

#in the BERT tutorial mode, there are a few steps:
#1. prepare data for Bert
#2. finetune Bert
#3. extract Bert embedding
#4. prepare the emedding to LSTM, which takes in variable input length, gotta do padding and masking and stuff. 


In [ ]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

#from torchtext import datasets

#train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
#example = next(iter(test_data))

In [ ]:
#field = {'date':('date',TEXT),'text_split':('text_split',TEXT),'change_label':('change_label',LABEL)}
#maybe i'll leave out the date first. 
field = {'text_split':('text_split',TEXT),'change_label':('change_label',LABEL)}
#field = {'date':('date',TEXT), 'change_label':('change_label',LABEL)}
#field = {'change_label':('change_label',LABEL)}

#I need to do something about the 'text_split' field with this tokenizer, the processing might to be done before hand. 

In [ ]:
train_1, val_1 = data.TabularDataset.splits(path='/content/gdrive/My Drive/Crypto/',train='train_save.csv',validation='val_save.csv',format='csv',fields=field)

In [ ]:
#this step is apparently not needed if my sentence is not gazillion long. 
import sys
import csv

csv.field_size_limit(sys.maxsize)


131072

In [ ]:
print (val_1[0].__dict__.keys())

dict_keys(['text_split', 'change_label'])


In [ ]:
print (val_1[4600].__dict__.values())

dict_values([['card', 'will', 'lead', 'to', 'easier', 'bitcoin', 'adoption', 'bitcoin', 'se', 'dos', 'us', 'as', 'hacker', 'over', 'million', 'worth', 'of', 'stolen', 'eth', 'to', 'unknown', 'wallet', 'btc', 'eth', 'xrp', 'the', 'key', 'to', 'eos', 'adoption', 'is', 'ram', 'on', 'na', 'din', 'me', 'wo', 'one', 'word', 'ho', 'in', 'bitcoin', 'make', 'lasting', 'revival', 'unlikely', 'from', 'want', 'to', 'win', 'free', 'register', 'with', 'only', 'via', 'this', 'is', 'project', 'that', 'your', 'participation', 'this', 'project', 'should', 'undoubtedly', 'interest', 'both', 'and', 'ordinary', 'eth', 'btc', 'ltc', 'bitcoin', 'btc', 'current', 'price', 'hour', 'days', 'why', 'apple', 'card', 'can', 'compete', 'with', 'bitcoin', 'top', 'bitcoin', 'sa', 'from', 'to', 'and', 'beyond', 'our', 'subscription', 'less', 'than', 'cup', 'of', 'coffee', 'here', 'are', 'the', 'monthly', 'day', 'yearly', 'off', 'day', 'is', 'the', 'real', 'money', 'the', 'fed', 'is', 'all', 'about', 'the', 'people', 'f

In [ ]:
df= pd.read_csv('/content/gdrive/My Drive/Crypto/val_save.csv')
df[4600:4603]

,text_split,change_label
4600,card will lead to easier bitcoin adoption bitc...,0
4601,resistant cryptocurrency feel free to refer to...,0
4602,could easily surpass gold in mike and it to pa...,0


In [ ]:
next(iter(train_1)).change_label
#so the text_split part, is a list. of al the tokens i recon, and '[' is even tokenized as well. 
#now it looks somewhat normal. 

'1'

In [ ]:
#now let's try out the build vocabulary part

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_1, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_) # how to initialize unseen words not in glove

LABEL.build_vocab(train_1)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                          
 99%|█████████▉| 397824/400000 [00:16<00:00, 23257.58it/s]

In [ ]:
TEXT.pad_token

'<pad>'

In [ ]:
#check the vocba size
len(TEXT.vocab)

17909

In [ ]:
#so...now that I've built the vocab, how do I check? obviously this vocab will not be very crypto currency friendly. 
#so ideally it should also be fine tuned. but let's get on with it for now. 

In [ ]:
BATCH_SIZE = 64

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cuda'



In [ ]:
torch.cuda.is_available()

True

In [ ]:
train_iterator, val_iterator = data.BucketIterator.splits(
    (train_1,val_1), 
    batch_size = BATCH_SIZE,
    #sort_within_batch = True,
    sort_key=lambda x: len(x.text_split), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False,
    device = device)

#error msg: Example' object has no attribute 'sort_key'
#stackoverflow says fields are not passed in the same order as they are in the csv/tsv file
#tried to convert labels from true/false to 1/0, didn't solve the error. 
#check values, nothing is null. how to solve this buuuuug. HALP!!!

#...ok, I don't need to split, if, i'm only iterating over train_l data...ok work! 

In [ ]:
# train_iterator = data.BucketIterator.splits(
#     train_1, 
#     batch_size = BATCH_SIZE,
#     #sort_within_batch = True,
#     sort_key=lambda x: len(x.text_split), # the BucketIterator needs to be told what function it should use to group the data.
#     sort_within_batch=False,
#     device = device)

# train_iterator[64].__dict__

In [ ]:
#seems like the sequnce is kept, which is just what I want, great. 

# train_iterator[0].dataset.text_split[0:10]

# Get into the model

In [ ]:
##now, let's get into the modeling part 
#1. defining the model
#2. writting out the training loop 


In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
      super().__init__()
      self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
      self.rnn = nn.LSTM(embedding_dim,
                         hidden_dim,
                         num_layers=n_layers,
                         bidirectional=bidirectional,
                         dropout=dropout)
      self.fc = nn.Linear(hidden_dim*2, output_dim)
      self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
      embedded = self.dropout(self.embedding(text))
      
      packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted = False) #added the enforce_sorted = false, debug
      packed_output, (hidden, cell) = self.rnn(packed_embedded)

      output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)#btw this is not used here i think.

      hidden = self.dropout(torch.cat((hidden[-2,:,:],hidden [-1,:,:]),dim =1 ))

      return self.fc(hidden)


      

In [ ]:
INPUT_DIM=len(TEXT.vocab)
EMBEDDING_DIM=100
HIDDEN_DIM=256
OUTPUT_DIM = 1
N_LAYERS=2
BIDIRECTIONAL=True
DROPOUT = 0.5
PAD_IDX= TEXT.vocab.stoi[TEXT.pad_token]

#i really donno what's the pad_token thing here...or how to use/where to learn vocab.stoi 
#TEXT.pad_token is just the dictionary key to the padding token, which is <pad> here.
#and TEXT.vocab.stoi['<pad>'] gives the padding value, 1. 
model = RNN(INPUT_DIM,EMBEDDING_DIM,HIDDEN_DIM,OUTPUT_DIM,N_LAYERS,BIDIRECTIONAL,DROPOUT,PAD_IDX)




In [ ]:
#checking trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,101,557 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([17909, 100])


In [ ]:
#copying the pretrained weights into the embedding. 
model.embedding.weight.data.copy_(pretrained_embeddings)


#i'm a little bit confused about this part for sure..why copying weights? then the embeddings is what? 
#so embedding layer is just like a dictionary with word index and as the key, values are just hte "weights"
#if i dont'wnat the weigts to be touched, I can specify in the model as
# print("Do not finetune word embedding layer.")
#             self.emb.weight.requires_grad = False


tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.8403,  0.1327, -0.6158,  ...,  0.7602, -0.2352,  0.3349],
        ...,
        [ 1.1097,  0.3746, -0.3882,  ..., -0.4966,  0.2572, -0.9995],
        [ 0.5207,  0.5319, -0.2065,  ...,  0.3557, -0.5426, -0.3839],
        [-0.5879,  0.0165, -0.6412,  ...,  0.1248,  0.0362, -0.3741]])

In [ ]:
#this step will set the <unk> and <pad> to be 0, and make them irrelevant for the model
#is this good though? the pretrained model is not on crypto words, which means the model WILL ignore crypto words
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.8403,  0.1327, -0.6158,  ...,  0.7602, -0.2352,  0.3349],
        ...,
        [ 1.1097,  0.3746, -0.3882,  ..., -0.4966,  0.2572, -0.9995],
        [ 0.5207,  0.5319, -0.2065,  ...,  0.3557, -0.5426, -0.3839],
        [-0.5879,  0.0165, -0.6412,  ...,  0.1248,  0.0362, -0.3741]])


# Now Train the Model

In [ ]:
#choose optimizer 
import torch.optim as optim

optimizer = optim.Adam(model.parameters(),weight_decay=0.001)

In [ ]:
#moving to GPU. 
criterion = nn.BCEWithLogitsLoss()

model= model.to(device)
criterion = criterion.to(device)

In [ ]:
device

'cuda'

In [ ]:
#defining accuracy metrics
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
#note, here we seperate the text and text.length as they are tuple

def train(model,iterator,optimizer, criterion):
  epoch_loss = 0 
  epoch_acc = 0

  model.train() #so that droput function works 
  for batch in iterator: 
    optimizer.zero_grad()#why? ok, to not accumulate un-intended gradient i guess.

    text, text_lengths = batch.text_split
    #predictions = model(text, text_lengths).squeeze(1) #why the fk do i need to move the length to cpu. 
    predictions = model(text, text_lengths.cpu()).squeeze(1) #what's the squeeze, what does squeeze do? pack np.array to be lower dimension?
    loss = criterion(predictions, batch.change_label)

    acc = binary_accuracy(predictions, batch.change_label)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator) , epoch_acc / len(iterator)



In [ ]:
def evaluate(model, iterator, criterion):

  epoch_loss = 0 
  epoch_acc = 0

  model.eval() #so that droput function DONT ACTIVATE

  with torch.no_grad():
  #is this what we do for evaluation always? 
  #answer: "with torch.no_grad()" temporarily set all the requires_grad flag to false.
    for batch in iterator: 
      text, text_lengths = batch.text_split
      #predictions = model(text, text_lengths).squeeze(1) #what's the squeeze, what does squeeze do? pack np.array to be lower dimension?
      predictions = model(text, text_lengths.cpu()).squeeze(1)
      loss = criterion(predictions, batch.change_label)

      acc = binary_accuracy(predictions, batch.change_label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(iterator) , epoch_acc / len(iterator)





In [ ]:
#define a timing function 
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
#now try to train yea
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 28s
	 Train Loss: 0.688 | Train Acc: 54.93%
	 Val. Loss: 0.680 |  Val. Acc: 58.46%
Epoch: 02 | Epoch Time: 0m 29s
	 Train Loss: 0.688 | Train Acc: 54.93%
	 Val. Loss: 0.682 |  Val. Acc: 58.46%
Epoch: 03 | Epoch Time: 0m 28s
	 Train Loss: 0.688 | Train Acc: 54.92%
	 Val. Loss: 0.681 |  Val. Acc: 58.46%
Epoch: 04 | Epoch Time: 0m 28s
	 Train Loss: 0.688 | Train Acc: 54.90%
	 Val. Loss: 0.682 |  Val. Acc: 58.46%
Epoch: 05 | Epoch Time: 0m 29s
	 Train Loss: 0.688 | Train Acc: 54.91%
	 Val. Loss: 0.682 |  Val. Acc: 58.46%
